In [1]:
import os
from os import listdir
from os import listdir
from os.path import isfile, join
from mwviews.utilities import aggregate
from mwviews import page_view_count
import gzip
import pandas as pd
from urllib.parse import quote, unquote

# Paths

In [2]:
files_path = "/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/"

# Getting file paths

In [3]:
def get_files(path):
    """ Returns a list of files in a directory
        Input parameter: path to directory
    """
    mypath = path
    complete = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    return complete

In [4]:
lst = get_files(files_path)

In [5]:
file_paths = [files_path+f for f in lst]

# Loading files

In [6]:
hour_files = [aggregate.open_hour_file(f) for f in file_paths[:10]]

In [7]:
hour_files

[<_io.TextIOWrapper name='/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090217-070000.gz' encoding='utf-8'>,
 <_io.TextIOWrapper name='/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090130-190000.gz' encoding='utf-8'>,
 <_io.TextIOWrapper name='/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090531-020000.gz' encoding='utf-8'>,
 <_io.TextIOWrapper name='/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20091011-160000.gz' encoding='utf-8'>,
 <_io.TextIOWrapper name='/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz' encoding='utf-8'>,
 <_io.TextIOWrapper name='/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090918-030000.gz' encoding='utf-8'>,
 <_io.TextIOWrapper name='/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090805-160000.gz' encoding='utf-8'>,

# Preparing output file

In [8]:
# output_file = open("proba", "w")

# Aggregating

In [9]:
# aggregate.run(hour_files,set(["en"]),output_file,False)

# Filtering with Gzip

In [10]:
def filter_gzip(path_old, path_new):
    fo = gzip.open('output.gz', 'w')
    f = gzip.open(file_paths[6], 'rb')
    for line in f.readlines():
        #     if line.split(" ")[0] == "en".encode():
        if line.decode().split(" ")[0] == "en":
            fo.write(line)
    f.close()
    fo.close()
    print("{} > {}, DONE! ".format(path_old, path_new))

In [11]:
# %timeit filter_gzip(file_paths[4], "output.gz") # this filters down just to EN wikipedia

In [12]:
# %timeit filter_gzip(file_paths[4], "output.gz")

In [13]:
# file_paths

# Filtering with Pandas

In [14]:
# df = pd.read_csv("output.gz", sep = " ")

In [15]:
# df.columns = ["project", "name", "views", "size"]
# df.head()

In [16]:
df = pd.read_csv("../data/cohort/wiki_page_cohorts/2009.csv")
df.head()

,names_u,names_q
0,Nizamettin_Erkmen,Nizamettin_Erkmen
1,Claudio_Scajola,Claudio_Scajola
2,Yang_Ti-liang,Yang_Ti-liang
3,Mohammad_Natsir,Mohammad_Natsir
4,Carolyn_Maloney,Carolyn_Maloney


In [17]:
def qoute_lst(lst):
    """ Changes normal characters in names to %c4%87 because, for example:
        'Slavica_Đukić_Dejanović' > 'Slavica_%C4%90uki%C4%87_Dejanovi%C4%87'
    """
    return [quote(name) for name in lst]

# Input params

In [18]:
s_politicians = df["names_q"]
pol_df = df.drop(["names_u"], axis=1)
# s_politicians.name = "name"
# pol_df = pd.DataFrame(s_politicians)
# s_politicians

In [19]:
# %timeit filter_gzip_pd(file_paths[4], "output.gz",s_politicians)

# Merge variations 

Faster than isin() and faster when filtering for project is done before the matching

In [20]:
def filter_gzip_pd_a(path_old, path_new, names, method="merge"):
    
    df = pd.read_csv(path_old, sep=" ")
    df.columns = ["project", "name", "views", "size"]
    df = df.drop(["size"], axis=1)
    if method == "merge":
        names.name="name"
        pol_df = pd.DataFrame(s_politicians)
#         df = df.set_index("name",drop=True)
#         df = df.join(names, how='inner')
        df = df.merge(pol_df, on=["name"])
    elif method == "list":
        df = df[df["name"].isin(names)]
    df = df[df["project"] == "en"]
    df = df.drop(["project"], axis=1)
#     print(df.head())
#     print(df.shape)
    df.to_csv(path_new, sep=" ",compression="gzip", index=False, header=False)
    print("{} > {}, DONE! ".format(path_old, path_new))

In [21]:
%timeit filter_gzip_pd_a(file_paths[4], "output.gz",s_politicians)

/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
8.17 s ± 545 ms per loop (mean ± std. dev. of 7 runs, 1 loop eac

In [23]:
def filter_gzip_pd_b(path_old, path_new, names, method="merge"):
    
    df = pd.read_csv(path_old, sep=" ")
    df.columns = ["project", "name", "views", "size"]
    df = df[df["project"] == "en"]
    df = df.drop(["size","project"], axis=1)
#     df = df.drop(["project"], axis=1)
    if method == "merge":
        names.name="name"
        pol_df = pd.DataFrame(s_politicians)
#         df = df.set_index("name",drop=True)
#         df = df.join(names, how='inner')
        df = df.merge(pol_df, on=["name"])
    elif method == "list":
        df = df[df["name"].isin(names)]
#     print(df.head())
#     print(df.shape)
    df.to_csv(path_new, sep=" ",compression="gzip", index=False, header=False)
    print("{} > {}, DONE! ".format(path_old, path_new))

In [24]:
%timeit filter_gzip_pd_b(file_paths[4], "output.gz",s_politicians)

/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
6.66 s ± 402 ms per loop (mean ± std. dev. of 7 runs, 1 loop eac

# Isin variations

In the case of using isin(), two things are interesting, it works faster if: 
* numpy array is passed rather than a series
* filtering by project is done after rather than before the matching 

In [25]:
%timeit filter_gzip_pd_a(file_paths[4], "output.gz",s_politicians, method="isin")

/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
15.3 s ± 688 ms per loop (mean ± std. dev. of 7 runs, 1 loop eac

In [26]:
%timeit filter_gzip_pd_b(file_paths[4], "output.gz",s_politicians, method="isin")

/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
16.8 s ± 647 ms per loop (mean ± std. dev. of 7 runs, 1 loop eac

In [27]:
lst = s_politicians.values

In [28]:
%timeit filter_gzip_pd_a(file_paths[4], "output.gz",lst, method="isin")

/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
16.7 s ± 29.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop ea

In [29]:
%timeit filter_gzip_pd_b(file_paths[4], "output.gz",lst, method="isin")

/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
16.4 s ± 57.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop ea

# Join vs Merge

merge is faster and more convinient

In [30]:
df = pd.read_csv(file_paths[4], sep=" ",index_col=1)
df.columns = ["project", "views", "size"]
df = df[df["project"] == "en"]
df = df.drop(["size","project"], axis=1)

In [31]:
# type(df.index.values.)

In [32]:
# %timeit df = df.merge(pol_df, on=["name"])


In [37]:
s_politicians.index = s_politicians.values
s_politicians
df.join(s_politicians, how='inner').head()

,views,name
%27Abd_al-Ilah,1,%27Abd_al-Ilah
%C3%81ine_Brady,1,%C3%81ine_Brady
%C3%81lvaro_Arz%C3%BA,2,%C3%81lvaro_Arz%C3%BA
%C3%81lvaro_Colom,64,%C3%81lvaro_Colom
%C3%81lvaro_Cunhal,1,%C3%81lvaro_Cunhal


In [38]:
%timeit df.join(s_politicians, how='inner')

4.07 s ± 88 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
df = pd.read_csv(file_paths[4], sep=" ")
df.columns = ["project", "name" ,"views", "size"]
df = df[df["project"] == "en"]
df = df.drop(["size","project"], axis=1)
x = pol_df
x.columns = ["name"]

In [40]:
%timeit df.merge(pol_df, on=["name"])

719 ms ± 4.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Final 

In [42]:
pol_df.columns = ["name"]

In [41]:
def parse(path_old, path_new, names_df):
    """ Reads file, eliminates unneeded data, filters for project "en" and sspecified names
    """
    df = pd.read_csv(path_old, sep=" ")
    df.columns = ["project", "name", "views", "size"]
    df = df[df["project"] == "en"]
    df = df.drop(["size","project"], axis=1)
    df = df.merge(names_df, on=["name"])
    df.to_csv(path_new, sep=" ",compression="gzip", index=False, header=False)
    print("{} > {}, DONE! ".format(path_old, path_new))

In [45]:
%timeit parse(file_paths[4],'output.gz', pol_df)

/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
/bigdata/vujovic/Wikipedia-Politician-Network/pageviews/files/2009/pagecounts-20090513-090000.gz > output.gz, DONE! 
6.32 s ± 38.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop ea

# Group by

In [ ]:
# filter_gzip_pd(file_paths[1000], "output4.gz",l_politicians, method="list")
# filter_gzip_pd(file_paths[100], "output5.gz",l_politicians, method="list")
# filter_gzip_pd(file_paths[10], "output6.gz",l_politicians, method="list")
# filter_gzip_pd(file_paths[1098], "output7.gz",l_politicians, method="list")
# filter_gzip_pd(file_paths[150], "output8.gz",l_politicians, method="list")
# filter_gzip_pd(file_paths[4658], "output9.gz",l_politicians, method="list")
# filter_gzip_pd(file_paths[7000], "output10.gz",l_politicians, method="list")

In [ ]:
dfs = []
for i in range(1,11):
    df = pd.read_csv("output{}.gz".format(i), sep=" ")
    df.columns = ["name", "views"]
    dfs.append(df)
ten_dfs = pd.concat(dfs)

In [ ]:
ten_dfs.head()

In [ ]:
df = ten_dfs.groupby(by="name")["views"].sum()
df.index

In [1]:
import pandas as pd

In [5]:
pd.read_csv("files/2009/pagecounts-20090101-000000.gz", sep=" ")

,ar.s,%D9%85%D9%82%D8%AF%D9%85%D8%A9_%D8%A7%D8%A8%D9%86_%D8%A7%D9%84%D8%B5%D9%84%D8%A7%D8%AD/%D8%A7%D9%84%D9%86%D9%88%D8%B9_%D8%A7%D9%84%D8%AB%D8%A7%D9%85%D9%86_%D9%88%D8%A7%D9%84%D8%AE%D9%85%D8%B3%D9%88%D9%86,1,10595
0,ar,%D8%A3%D9%87%D9%84_%D8%A7%D9%84%D8%A8%D9%8A%D8%AA,1,15365
1,ar,%D8%A7%D9%84%D8%B5%D9%81%D8%AD%D8%A9_%D8%A7%D9...,1,20325
2,ar,%D8%A8%D8%B1%D8%AC_%D8%A7%D9%84%D8%B3%D8%B1%D8...,1,21010
3,ar,%D8%AE%D8%A7%D8%B5:%D9%85%D8%A7%D8%B0%D8%A7_%D...,1,6660
4,ar,%D8%B9%D8%A8%D8%AF_%D8%A7%D9%84%D8%B9%D8%B2%D9...,1,30030
5,ar,"خاص:بحث/Columbus,_Ohio",1,21141
6,be-x-old,%D0%93%D1%96%D0%BC%D0%BD_%D0%92%D1%83%D0%B3%D0...,1,13320
7,be,%D0%93%D0%BE%D1%80%D0%B0%D0%B4_%D0%90%D0%BA%D1...,1,9286
8,beta.v,Special:RecentChangesLinked/%EB%AA%A8%EC%97%90...,1,5215
9,bg,%D0%A0%D0%BE%D1%81%D1%82%D0%B8%D1%81%D0%BB%D0%...,1,709
